In [10]:
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np

from processing.networks.unetVariants import UNetNoPad2
from postprocessing.visu_utils import _aligned_colorbar
import postprocessing.cmap_jp
from postprocessing.metric_connectivity import calc_connectivity

In [2]:
%reload_ext autoreload
%autoreload 2
%load_ext line_profiler

In [3]:
run_id = 2
threshold = 10.7
args = {"cmap" : "jp_wbRow_neon",
            "interpolation" :"none",
            "vmin": 10.6,
        }
args_01 = {"cmap": "binary"}

# Data prep + Calculation of connectivity

In [ ]:
# Label, naive
model_path = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/naive_approach_unetnopad")
data_path = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_pki outputs_t")
id_mat_ids = 2
model = UNetNoPad2(in_channels=3, out_channels=1, depth=3, init_features=32, kernel_size=5).float()

conn_label, conn_naive = calc_connectivity(model_path, data_path, run_id, model, id_mat_ids, threshold)
print("Label: ",conn_label["ratio"])
print("Naive: ",np.round(conn_naive["ratio"],4))

In [ ]:
# T_sim(sim)
model_path = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/BEST_predict_T_add_s_outer")
data_path = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_ixydk+s_outer outputs_t")
id_mat_ids = 0
model = UNetNoPad2(in_channels=6, out_channels=1, depth=4, init_features=32, kernel_size=4).float()

_, conn_simsim = calc_connectivity(model_path, data_path, run_id, model, id_mat_ids, threshold)
print("T_sim(sim): ",np.round(conn_simsim["ratio"],4))


In [ ]:
# model trained on sim, applied to pred
model_path = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/BEST_predict_T_add_s_outer")
data_path = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_ixydk+s_outer outputs_t prep_with_BEST_predict_v_v4")
id_mat_ids = 0
model = UNetNoPad2(in_channels=6, out_channels=1, depth=4, init_features=32, kernel_size=4).float()
_, conn_simpred = calc_connectivity(model_path, data_path, run_id, model, id_mat_ids, threshold)
print("T_sim(pred): ",np.round(conn_simpred["ratio"],4))

In [ ]:
# model trained on pred, applied to pred
model_path = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/predict_T_from_s_outer based_on_predict_v")
id_mat_ids = 0
data_path = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_ixydk+s_outer outputs_t prep_with_BEST_predict_v_v4")
model = UNetNoPad2(in_channels=6, out_channels=1, depth=4, init_features=32, kernel_size=4).float()
_, conn_predpred = calc_connectivity(model_path, data_path, run_id, model, id_mat_ids, threshold)
print("T_pred(pred): ",np.round(conn_predpred["ratio"],4))

# Visualisation

In [ ]:
plt.figure(figsize=(15,25))
for id, (name, prop) in enumerate([("Label", conn_label), ("Naive", conn_naive), ("T_sim(sim)", conn_simsim), ("T_sim(pred)", conn_simpred), ("T_pred(pred)", conn_predpred)]):
    plt.subplot(5,3,id*3+1)
    plt.title(name)
    plt.imshow(prop["field"].T, **args) #[0:100,2200:2300])
    _aligned_colorbar()
    plt.subplot(5,3,id*3+2)
    plt.title("Connected cells")
    plt.imshow(prop["connectivity"].T, **args_01)
    plt.subplot(5,3,id*3+3)
    plt.title("Unconnected cells")
    plt.imshow(prop["unconnected_cells"].T, **args_01)
plt.show()

# OLD

# Flood fill - comparison numpy, torch
numpy

In [ ]:
# def in_bounds(curr_pos, size):
#     tmp = (curr_pos >= 0) & (curr_pos < size)
#     return tmp[0] & tmp[1]
 
# def select_in_bounds(curr_pos, size):
#     return curr_pos.T[in_bounds(curr_pos, size)].T
 
# def filter_visited(curr_pos, visited, field):
#     return curr_pos.T[(~visited[curr_pos[0],curr_pos[1]] & field[curr_pos[0],curr_pos[1]])].T
 
# def step(active_indices, visited, field):
#     size = visited.shape[0]
#     visited[active_indices[0],active_indices[1]] = True
#     up = active_indices.copy()
#     up[1] += 1
#     down = active_indices.copy()
#     down[1] -= 1
#     right = active_indices.copy()
#     right[0] += 1
#     left = active_indices.copy()
#     left[0] -= 1
#     up = select_in_bounds(up, size)
#     down = select_in_bounds(down, size)
#     left = select_in_bounds(left, size)
#     right = select_in_bounds(right, size)
#     up = filter_visited(up, visited, field)
#     visited[up[0],up[1]] = True
#     down = filter_visited(down, visited, field)
#     visited[down[0],down[1]] = True
#     left = filter_visited(left, visited, field)
#     visited[left[0],left[1]] = True
#     right = filter_visited(right, visited, field)
#     visited[right[0],right[1]] = True
 
#     new_indices = np.concatenate((up, down, left, right), axis=1)

#     return new_indices
 
# def flood_fill(active_indices, field):
#     visited=np.zeros_like(field,dtype=bool)
#     i = 0
#     while len(active_indices[0]) > 0:
#         i += 1
#         active_indices = step(active_indices, visited, field)
#     return visited

# def test_flood_fill():
#     np.random.seed(0)
#     size = 2560
#     n_start = 100
#     field_to_fill = np.ones((size, size), dtype=bool)
#     active_indices = np.random.randint(0, size - 1, (2, n_start))
#     flood_fill(active_indices, field_to_fill)

# def connectivity_field_flood(mat_ids_unnormed, mask_output):
#     hps = np.where(mat_ids_unnormed == 2)
#     hps = np.array(hps)
#     connectivity_field  = flood_fill(hps, mask_output[0].cpu().numpy())

#     unconnected_cells = connectivity_field ^ np.array(mask_output[0])
#     connected_cells = np.sum(np.array(mask_output[0]))
#     # print("Ratio: ", np.sum(unconnected_cells)/connected_cells, ", unconnected cells: ", np.sum(unconnected_cells), ", connected cells: ", connected_cells)
#     return connectivity_field, unconnected_cells, connected_cells

# # %lprun -f step connectivity_label, unconn_cells_label, conn_cells_label            = connectivity_field_flood(inputs_naive[2], mask_label)

# %timeit connectivity_field_flood(inputs_naive[2], mask_label)
# %timeit connectivity_field_flood(inputs_naive[2], mask_output_naive)
# %timeit connectivity_field_flood(inputs_sim[0], mask_output_simsim)
# %timeit connectivity_field_flood(inputs_pred[0], mask_output_simpred)
# %timeit connectivity_field_flood(inputs_pred[0], mask_output_predpred)


In [8]:
## TORCH

# def in_bounds(curr_pos, size):
#     tmp = (curr_pos >= 0) & (curr_pos < size)
#     return tmp[0] & tmp[1]
 
# def select_in_bounds(curr_pos, size):
#     return curr_pos.T[in_bounds(curr_pos, size)].T
 
# def filter_visited(curr_pos, visited, field):
#     return curr_pos.T[(~visited[curr_pos[0],curr_pos[1]] & field[curr_pos[0],curr_pos[1]])].T
 
# def step(active_indices, visited, field):
#     size = visited.shape[0]
#     visited[active_indices[0],active_indices[1]] = True
#     up = active_indices.clone()
#     up[1] += 1
#     down = active_indices.clone()
#     down[1] -= 1
#     right = active_indices.clone()
#     right[0] += 1
#     left = active_indices.clone()
#     left[0] -= 1
#     up = select_in_bounds(up, size)
#     down = select_in_bounds(down, size)
#     left = select_in_bounds(left, size)
#     right = select_in_bounds(right, size)
#     up = filter_visited(up, visited, field)
#     visited[up[0],up[1]] = True
#     down = filter_visited(down, visited, field)
#     visited[down[0],down[1]] = True
#     left = filter_visited(left, visited, field)
#     visited[left[0],left[1]] = True
#     right = filter_visited(right, visited, field)
#     visited[right[0],right[1]] = True
 
#     new_indices = torch.concat((up, down, left, right), dim=1)

#     return new_indices
 
# def flood_fill(active_indices, field):
#     visited=torch.zeros_like(field,dtype=bool).to("cuda")
#     i = 0
#     while len(active_indices[0]) > 0:
#         i += 1
#         active_indices = step(active_indices, visited, field)
#     return visited

# def connectivity_field_flood(mat_ids_unnormed, mask_output):
#     mask_output = mask_output.to("cuda")
#     hps = torch.where(mat_ids_unnormed == 2)
#     hps = torch.stack(list(hps), dim=0).to("cuda")
#     connectivity_field  = flood_fill(hps, mask_output[0])

#     unconnected_cells = connectivity_field ^ mask_output[0]
#     connected_cells = torch.sum(mask_output[0])
#     # print("Ratio: ", np.sum(unconnected_cells)/connected_cells, ", unconnected cells: ", np.sum(unconnected_cells), ", connected cells: ", connected_cells)

#     return connectivity_field, unconnected_cells, connected_cells
# %lprun -f step connectivity_field_flood(inputs_naive[2], mask_label)
# print("Ratio: ", np.sum(unconn_cells_label)/conn_cells_label, ", unconnected cells: ", np.sum(unconn_cells_label), ", connected cells: ", conn_cells_label)


### torch + cpu
1.3 s ± 106 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.0 , unconnected cells:  0 , connected cells:  2147604
1.07 s ± 3.93 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.3965575866641409 , unconnected cells:  837371 , connected cells:  2111600
1.38 s ± 45.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.009183051959207258 , unconnected cells:  17052 , connected cells:  1856899
1.2 s ± 64.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.004250811282885087 , unconnected cells:  6970 , connected cells:  1639687
1.29 s ± 35.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.03488861513191664 , unconnected cells:  61173 , connected cells:  1753380

### torch + gpu
1.56 s ± 2.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.0 , unconnected cells:  0 , connected cells:  2147604
1.7 s ± 1.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.3965575866641409 , unconnected cells:  837371 , connected cells:  2111600
1.9 s ± 8.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.009183051959207258 , unconnected cells:  17052 , connected cells:  1856899
1.31 s ± 1.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.004250811282885087 , unconnected cells:  6970 , connected cells:  1639687
1.28 s ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.03488861513191664 , unconnected cells:  61173 , connected cells:  1753380

### numpy
371 ms ± 2.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.0 , unconnected cells:  0 , connected cells:  2147604
262 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.3965575866641409 , unconnected cells:  837371 , connected cells:  2111600
345 ms ± 989 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.009183051959207258 , unconnected cells:  17052 , connected cells:  1856899
291 ms ± 4.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.004250811282885087 , unconnected cells:  6970 , connected cells:  1639687
302 ms ± 10.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Ratio:  0.03488861513191664 , unconnected cells:  61173 , connected cells:  1753380

# Data and model loading

In [ ]:
# model_path = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/naive_approach_unetnopad")
# data_path = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_pki outputs_t")

# model = UNetNoPad2(in_channels=3, out_channels=1, depth=3, init_features=32, kernel_size=5).float()
# model.load(model_path)
# data = DataPoint(data_path, idx=2)
# inputs_naive, label = data[0]
# output_naive = model.infer(inputs_naive.unsqueeze(0))
# inputs_naive, label, output_naive = reverse_norm_one_dp(inputs_naive, label, output_naive, data.norm)

# model_path_sim = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/BEST_predict_T_add_s_outer")
# data_path_sim = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_ixydk+s_outer outputs_t")

# model_sim = UNetNoPad2(in_channels=6, out_channels=1, depth=4, init_features=32, kernel_size=4).float()
# model_sim.load(model_path_sim)
# data_sim = DataPoint(data_path_sim, idx=2)
# inputs_sim, label_sim = data_sim[0]
# output_simsim = model_sim.infer(inputs_sim.unsqueeze(0))
# inputs_sim, label_sim, output_simsim = reverse_norm_one_dp(inputs_sim, label_sim, output_simsim, data_sim.norm)

# model_path_pred = Path("/home/pelzerja/pelzerja/test_nn/1HP_NN/runs/allin1/paper24 finals/predict_T_from_s_outer based_on_predict_v")
# data_path_pred = Path("/scratch/sgs/pelzerja/datasets_prepared/allin1/dataset_giant_100hp_varyK inputs_ixydk+s_outer outputs_t prep_with_BEST_predict_v_v4")

# model_pred = UNetNoPad2(in_channels=6, out_channels=1, depth=4, init_features=32, kernel_size=4).float()
# model_pred.load(model_path_pred)
# data_pred = DataPoint(data_path_pred, idx=2)
# inputs_pred, label_pred = data_pred[0]
# # model trained on sim, applied to pred
# output_simpred = model_sim.infer(inputs_pred.unsqueeze(0))
# # model trained on pred, applied to pred
# output_predpred = model_pred.infer(inputs_pred.unsqueeze(0))
# _, _, output_simpred = reverse_norm_one_dp(deepcopy(inputs_pred), deepcopy(label_pred), output_simpred, data_sim.norm)
# inputs_prep, label_pred, output_predpred = reverse_norm_one_dp(inputs_pred, label_pred, output_predpred, data_sim.norm)

# Data preparation

In [ ]:
# def unify_size(inputs, required_size):
#     # expect square
#     start = int((len(inputs[0,0]) - required_size)//2)
#     inputs = inputs[:,start:start+required_size, start:start+required_size]
#     return inputs

# required_size = min(len(output_naive[0,0]),  len(output_simsim[0,0]),  len(output_simpred[0,0]),  len(output_predpred[0,0]))
# print(required_size)
# # inputs_naive = unify_size(inputs_naive, required_size)
# inputs_sim = unify_size(inputs_sim, required_size)
# inputs_pred = unify_size(inputs_pred, required_size)

# # label = unify_size(label, required_size)
# # output_naive = unify_size(output_naive, required_size)
# output_simsim = unify_size(output_simsim, required_size)
# output_simpred = unify_size(output_simpred, required_size)
# output_predpred = unify_size(output_predpred, required_size)

# Data masking at threshold

In [ ]:
# # label
# mask_label = label > threshold
# masked_label = label[mask_label]

# # naive
# mask_output_naive = output_naive > threshold
# masked_output_naive = output_naive[mask_output_naive]

# # T_sim(sim)
# mask_output_simsim = output_simsim > threshold
# masked_output_simsim = output_simsim[mask_output_simsim]

# # T_sim(pred)
# mask_output_simpred = output_simpred > threshold
# masked_output_simpred = output_simpred[mask_output_simpred]

# # T_pred(pred)
# mask_output_predpred = output_predpred > threshold
# masked_output_predpred = output_predpred[mask_output_predpred]

In [ ]:
# connectivity_label, unconn_cells_label, conn_cells_label            = connectivity_field_flood(inputs_naive[2], mask_label)
# print("Ratio: ", np.sum(unconn_cells_label)/conn_cells_label, ", unconnected cells: ", np.sum(unconn_cells_label), ", connected cells: ", conn_cells_label)
# connectivity_naive, unconn_cells_naive, conn_cells_naive            = connectivity_field_flood(inputs_naive[2], mask_output_naive)
# print("Ratio: ", np.sum(unconn_cells_naive)/conn_cells_naive, ", unconnected cells: ", np.sum(unconn_cells_naive), ", connected cells: ", conn_cells_naive)
# connectivity_simsim, unconn_cells_simsim, conn_cells_simsim         = connectivity_field_flood(inputs_sim[0], mask_output_simsim)
# print("Ratio: ", np.sum(unconn_cells_simsim)/conn_cells_simsim, ", unconnected cells: ", np.sum(unconn_cells_simsim), ", connected cells: ", conn_cells_simsim)
# connectivity_simpred, unconn_cells_simpred, conn_cells_simpred      = connectivity_field_flood(inputs_pred[0], mask_output_simpred)
# print("Ratio: ", np.sum(unconn_cells_simpred)/conn_cells_simpred, ", unconnected cells: ", np.sum(unconn_cells_simpred), ", connected cells: ", conn_cells_simpred)
# connectivity_predpred, unconn_cells_predpred, conn_cells_predpred   = connectivity_field_flood(inputs_pred[0], mask_output_predpred)
# print("Ratio: ", np.sum(unconn_cells_predpred)/conn_cells_predpred, ", unconnected cells: ", np.sum(unconn_cells_predpred), ", connected cells: ", conn_cells_predpred)
